In [ ]:
import numpy as np 
import pandas as pd 

import os


#FOLLOWING FUNCTION os.walk() GOES TO EACH DIRECTORY AND RETURNS A 3-TUPLE (DIRPATH,dirs in this dir, Files in this dir)


# for dirname, _, filenames in os.walk('/kaggle/input'): # '_' what does this mean
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
# ls=next(os.walk('/kaggle/input'))
# ls

In [ ]:
df=pd.read_csv("/kaggle/input/imaterialist-fashion-2020-fgvc7/train.csv")

In [ ]:
# type(df['EncodedPixels'][2]) #encoded pixels are stored as string

In [ ]:
import gc  #garbage collector
import sys 
import json
import glob
import random
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import itertools
from tqdm import tqdm

from imgaug import augmenters as iaa  #data augmentatoin (multiple representations of the same image')'
from sklearn.model_selection import StratifiedKFold, KFold


In [ ]:
datadir = Path('/kaggle/input/imaterialist-fashion-2020-fgvc7/')
modeldir = Path('/kaggle/working')

In [ ]:
NUM_CATS = 3
IMAGE_SIZE = 512

In [ ]:
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')    

!rm -rf .git # to prevent an error when the kernel is committed
!rm -rf images assets # to prevent displaying images at the bottom of a kernel

In [ ]:
os.chdir('/kaggle/working/Mask_RCNN')

In [ ]:
sys.path.append('/kaggle/working/Mask_RCNN')


In [ ]:
# sys.path is the list of directories and files pthon goes through looking for modules and files

In [ ]:
# !pip install tensorflow==1.15.0rc0 #need that because Mask-RCNN version
# !pip install keras==2.1.5

import tensorflow
print(tensorflow.__version__)
import keras
print(keras.__version__)

In [ ]:
#os.chdir() changed the working directory to Mask_RCNN

In [ ]:
from mrcnn.config import Config #need to have tensorflow 2.2.0 installed
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
# !wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5  #wget command in linux is used to download files from the internet
# !ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = 'mask_rcnn_coco.h5'


In [ ]:
class FashionConfig(Config):
    NAME = "fashion"
    NUM_CLASSES = 3 # +1 for the background class
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 # a memory error occurs when IMAGES_PER_GPU is too high
    
    BACKBONE = 'resnet50'
    
    IMAGE_MIN_DIM = IMAGE_SIZE   #config class uses these variables image_min_dims and image_max_dims, need to feed them the image size
    IMAGE_MAX_DIM = IMAGE_SIZE    
    IMAGE_RESIZE_MODE = 'none'
    
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 20
    
config = FashionConfig()
# config.display()


Making Dataset

In [ ]:
with open(datadir/'label_descriptions.json') as f:
    label_descriptions=json.load(f)
    
label_names=[]
for x in label_descriptions['categories']:#we'll only take the data for classes 0 and 10
    if((x['id']==0)&(x['id']==10)):
        label_names.append(x['name'])



In [ ]:
df1=df[(df['ClassId']==0)|(df['ClassId']==10)]

In [ ]:
image_df=df1.groupby('ImageId')['EncodedPixels','ClassId'].agg(lambda x:list(x))
size_df=df1.groupby('ImageId')['Height','Width'].mean()
image_df=image_df.join(size_df,on='ImageId')
image_df.reset_index(inplace=True)

In [ ]:
# image_df.head()

* Function to resize images

In [ ]:
from skimage import io

def resize_image(image_path):
    try:
        img = io.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_AREA)  
        return img
    except Exception as e:
        print(str(e))

In [ ]:
with open(datadir/'label_descriptions.json') as f:
    label_descriptions=json.load(f)
    
label_names=[]
for x in label_descriptions['categories']:#we'll only take the data for classes 0 and 10
    if((x['id']==0)|(x['id']==10)):
        label_names.append(x['name'])

In [ ]:
class FashionDataset(utils.Dataset):

    def __init__(self, df):
        super().__init__(self)
        
        # Add classes
        for i, name in enumerate(label_names):
            self.add_class("fashion", str(i), name)

                
        # Add images 
        for i, row in df.iterrows():
            if(row['ClassId'][0]==10):
                row['ClassId'][0]=1
            self.add_image("fashion", 
                           image_id=row.name, 
                           path=str(datadir/'train'/row['ImageId'])+'.jpg',
                           labels=row['ClassId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']
    def load_image(self, image_id):
        return resize_image(self.image_info[image_id]['path'])

    def load_mask(self, image_id):
        info = self.image_info[image_id]  #info refers to info of an image(height,width,labels,annotation..)
                
        mask = np.zeros((512, 512,1)), dtype=np.uint8)  #each mask will be the same shape as the image i.e. (512,512,1)
        labels = []
        
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(info['height']*info['width'], 0, dtype=np.uint8) #creating a mask of the size of the original image
            annotation = [int(x) for x in annotation.split(' ')]  #splitting and converting all entries to int
            
            
            #decoding the encoded mask
            for i, start_pixel in enumerate(annotation[::2]):
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1
            
            #converting it to a 2-D array, resizing it to the shape as that of the input
            sub_mask = sub_mask.reshape((info['height'], info['width']), order='F')
            sub_mask = cv2.resize(sub_mask, (512, 512), interpolation=cv2.INTER_NEAREST)
            
            mask[:, :, m] = sub_mask  #mth mask for that particular image
            labels.append(int(label))
            
        return mask, np.array(labels)

In [ ]:
dataset = FashionDataset(image_df)
dataset.prepare()


In [ ]:
# dataset.class_names   BG refers to background

In [ ]:
# len(label_names)

In [ ]:
for i in range(10):
    image_id = random.choice(dataset.image_ids)
    print(dataset.image_reference(image_id))
    
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    print(class_ids)
    visualize.display_top_masks(image, mask,class_ids,dataset.class_names, limit=2) #limit of what?

In [ ]:
FOLD = 0
N_FOLDS = 5

kf = KFold(n_splits=N_FOLDS, random_state=42, shuffle=True)
splits = kf.split(image_df) # ideally, this should be multilabel stratification

def get_fold():    
    for i, (train_index, valid_index) in enumerate(splits): #train_index and valid_index are arrays. iloc will return a df when multiple rows are passed/ returns a series object otherwise  
        if i == FOLD:
            return image_df.iloc[train_index], image_df.iloc[valid_index]
        
train_df, valid_df = get_fold()

train_dataset = FashionDataset(train_df)
train_dataset.prepare()

valid_dataset = FashionDataset(valid_df)
valid_dataset.prepare()

In [ ]:
# valid_df.shape 20% of the data is validation data

In [ ]:
train_segments = np.concatenate(train_df['ClassId'].values).astype(int)  #concatinating to a single list
print("Total train images: ", len(train_df))
print("Total train segments: ", len(train_segments)) #this len can be greater than no of train images since one image could have more than one class id associated

plt.figure(figsize=(12, 3))
values, counts = np.unique(train_segments, return_counts=True) #number of zeroes and ones
plt.bar(values, counts)
plt.xticks(values, label_names, rotation='vertical')
plt.show()

valid_segments = np.concatenate(valid_df['ClassId'].values).astype(int)
print("Total train images: ", len(valid_df))
print("Total validation segments: ", len(valid_segments))

plt.figure(figsize=(12, 3))
values, counts = np.unique(valid_segments, return_counts=True)
plt.bar(values, counts)
plt.xticks(values, label_names, rotation='vertical')
plt.show()



In [ ]:
LR = 1e-4
EPOCHS = [2, 6, 8]

import warnings 
warnings.filterwarnings("ignore")

In [ ]:

model = modellib.MaskRCNN(mode='training', config=config, model_dir=modeldir)

model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    'mrcnn_class_logits', 'mrcnn_bbox_fc', 'mrcnn_bbox', 'mrcnn_mask'])



Data Augmentation

In [ ]:
augmentation = iaa.Sequential([
    iaa.Fliplr(0.5) # only horizontal flip here
])

In [ ]:
%%time
model.train(train_dataset, valid_dataset,
            learning_rate=LR*2, # train heads with higher lr to speedup learning
            epochs=EPOCHS[0],
            layers='heads',
            augmentation=augmentation)

history = model.keras_model.history.history

In [ ]:
sample_df=pd.read_csv(DATA_DIR/'sample_submission.csv')

In [ ]:
history = model.keras_model.history.history

In [ ]:
class InferenceConfig(FashionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=modeldir)
model_path='/kaggle/working/fashion20200518T1903/mask_rcnn_fashion_0002.h5'
# assert model_path != '/kaggle/working/fashion20200517T2123/mask_rcnn_fashion_001.h5', "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
def to_rle(bits):
    rle = []
    pos = 0
    for bit, group in itertools.groupby(bits):
        group_list = list(group)
        if bit:
            rle.extend([pos, sum(group_list)])
        pos += len(group_list)
    return rle

In [ ]:
def refine_masks(masks, rois):
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    mask_index = np.argsort(areas)
    union_mask = np.zeros(masks.shape[:-1], dtype=bool)
    for m in mask_index:
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
        union_mask = np.logical_or(masks[:, :, m], union_mask)
    for m in range(masks.shape[-1]):
        mask_pos = np.where(masks[:, :, m]==True)
        if np.any(mask_pos):
            y1, x1 = np.min(mask_pos, axis=1)
            y2, x2 = np.max(mask_pos, axis=1)
            rois[m, :] = [y1, x1, y2, x2]
    return masks, rois

In [ ]:


# %%time
# sub_list = []
# missing_count = 0
# for i, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
#     image = resize_image(str(DATA_DIR/'test'/row['ImageId'])+'.jpg')
#     result = model.detect([image])[0]
#     if result['masks'].size > 0:
#         masks, _ = refine_masks(result['masks'], result['rois'])
#         for m in range(masks.shape[-1]):
#             mask = masks[:, :, m].ravel(order='F')
#             rle = to_rle(mask)
#             label = result['class_ids'][m] - 1
#             sub_list.append([row['ImageId'], ' '.join(list(map(str, rle))), label])
#     else:
#         # The system does not allow missing ids, this is an easy way to fill them 
#         sub_list.append([row['ImageId'], '1 1', 23])
#         missing_count += 1



In [ ]:
# for i in range(4):
# image_id = sample_df.sample()['ImageId'].values[0]
image_id='0002f5a0ebc162ecfb73e2c91e3b8f62'
image_path = str(datadir/'train'/image_id)+'.jpg'

img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

result = model.detect([resize_image(image_path)])
r = result[0]

if r['masks'].size > 0:
    masks = np.zeros((512, 512, r['masks'].shape[-1]), dtype=np.uint8)
    for m in range(r['masks'].shape[-1]):
        masks[:, :, m] = cv2.resize(r['masks'][:, :, m].astype('uint8'), 
                                    (512, 512), interpolation=cv2.INTER_NEAREST)

    y_scale = 512/IMAGE_SIZE
    x_scale = 512/IMAGE_SIZE
    rois = (r['rois'] * [y_scale, x_scale, y_scale, x_scale]).astype(int)

    masks, rois = refine_masks(masks, rois)
else:
    masks, rois = r['masks'], r['rois']
# mask2=np.zeros((512, 512))
# for j in range(masks.shape[2]):
#     mask2=np.maximum(masks[:,:,j],mask2)

#     masks2=np.maximum(masks[:,:,j],masks2)
# print(masks2)
# plt.imshow(masks2)
# visualize.display_instances(img, rois, masks, r['class_ids'], 
#                             ['bg']+label_names, r['scores'],
#                             title=image_id, figsize=(12, 12))
visualize.display_instances(resize_image(image_path), r['rois'], r['masks'], r['class_ids'], 
                            ['BG','Shirt','Dress'], r['scores'])
    

In [ ]:
plt.imshow(mask2)

In [ ]:
valid_df.head()